In [389]:
import os
import sys
from sys import *
import pandas as pd
import numpy as np
import cmd
import pexpect
from subprocess import *
from pymystem3 import Mystem
import pymystem3 as pm
import pymorphy2 as pm2
from functools import reduce
import re
from tabulate import tabulate

In [445]:
def homework_1(way):
    in_file = way
    out_file_1 = r'C:\\Users\\Ivanitskiy\\Documents\\CL\\out.txt'
    out_file_2 = r'C:\\Users\\Ivanitskiy\\Documents\\CL\\out2.txt'
    command_1 = r'mystem ' + in_file + ' '+ out_file_1
    command_2 = r'mystem -cg ' + in_file + ' '+ out_file_2
    call(pexpect.split_command_line(command_1))
    call(pexpect.split_command_line(command_2))
    df = pd.read_csv(out_file_1, sep = '{', lineterminator='}',header=None)
    df_temp = pd.read_csv(out_file_2, sep = '{', lineterminator='}',header=None)
    df.columns = ['word','spec']
    df_temp.columns = ['word','spec']
    df = df
    df_temp = df_temp
    df = pd.merge(df, df_temp, left_index=True, right_index=True)

    # Part 1

    ### Количество словоупотреблений
    print()
    count_of_usage = df.shape[0]
    print('Общее число словоупотреблений = %d' % count_of_usage)

    ### Количество словоформ
    print()
    def rep(x):
        temp = str(x).replace('\n','').replace('\ufeff','').replace('\r','').replace(',','').replace(';','').replace('...','').replace(r'.','')
        temp = temp.replace('—','').replace('?','').replace('!','').replace(r'.','').replace('\t','').replace(r'!','').replace(r'?','').replace(':','')
        temp = temp.replace('"','').replace('»','').replace('«','').replace('„','').replace('“','').replace(' ','').upper()
        return temp

    df['word_form'] = list(map(lambda x: rep(x),
                               df['word_x']))

    count_word_form = df['word_form'].nunique()
    print('Общее число словоформ = %d' % count_word_form)

    ### Число незнакомых слов
    print()
    m = pm.Mystem()
    def is_known_mystem(x):
        try:
            temp = x[0]['analysis'][0]['qual'] == 'bastard'
        except:
            temp = False
        return not temp

    f = open(in_file,'r',encoding='utf8')
    file_text = ''
    for line in f:
        file_text = file_text + str(line).replace('\ufeff','').replace('\n','')
    result_of_text_analyze = m.analyze(file_text)

    result_of_text_analyze_filter = []
    for i in result_of_text_analyze:
        if pd.Series(i['text']).isin([', ','. ','! ','? ','\n','\ufeff','„','“','«','»','-','!','?','... ',': «','...',
                                      ' «','», — ','!» ',':«','...“ ',
                                      '—','; ',': ','?» ',' ','.',':— ','.— ','?— ','!— ','...— ','— ',', — ',' — ']).values == False:
            result_of_text_analyze_filter.append(i)
    df['spec_by_python_lib_mystem'] = pd.Series(result_of_text_analyze_filter)

    analyzer = pm2.MorphAnalyzer()
    df['is_known_pymorphy2'] = list(map(lambda x: int(analyzer.word_is_known(x)), df['word_form']))
    df['is_known_mystem'] = list(map(lambda x: int(is_known_mystem(x)), df['spec_by_python_lib_mystem']))
    count_of_known_words_pymorphy = sum(df['is_known_pymorphy2'])
    count_of_known_words_mystem = sum(df['is_known_mystem'])
    print('Число слов знакомых словарю:\n\t* pymorphy\t%d\n\t* mystem\t%d' % (count_of_known_words_pymorphy, count_of_known_words_mystem))

    ### Количество уникальных лемм
    print()
    result_lexem = ''
    lexems = reduce(lambda x,y: y+x,list(map(lambda x: x+'|', df['spec_x'])))
    lexems_set = set(lexems.split('|')[:-1])
    # count of lexems with omonimia
    count_of_lexems_1 = len(lexems_set)
    # count of lexems without omonimia
    count_of_lexems_2 = df['spec_x'].nunique()
    print('Число лексем:\n\t* с учетом омонимии\t%d\n\t(если одной словоформе соответствуют несколько лексем, то они все учитываются)\n\t* без учета омонимии\t%d\n\t(при соответствии нескольким лексемам, смотрится совпадение всех предполагаемых лексем)'
          % (count_of_lexems_1, count_of_lexems_2))

    ### Коэффициент лексического богатства текста
    print()
    print('Коэффициент лексического богатства текста:\n\t* с учетом омонимии\t%f05\n\t(если одной словоформе соответствуют несколько лексем, то они все учитываются)\n\t* без учета омонимии\t%f05\n\t(при соответствии нескольким лексемам, смотрится совпадение всех предполагаемых лексем)'
          % (count_of_lexems_1/float(count_of_usage), count_of_lexems_2/float(count_of_usage)))

    # Part 2

    ### Абсолютная и относительная частота омонимичных словоформ
    print()
    print()
    def gr(x):
        try:
            return x['analysis'][0]['gr']
        except:
            return ''

    df['count_of_lexems'] = list(map(lambda x: len(x.split('|')), df['spec_x']))
    df['grammar'] = list(map(lambda x: gr(x).split('=')[0], df['spec_by_python_lib_mystem']))
    unchangeable = ['PR','CONJ','PART','ADVPRO','ADV']
    df['changeable'] = list(map(lambda x: int(not pd.Series(x).isin(unchangeable)[0]), df['grammar']))
    count_omonimes = sum(df['count_of_lexems'] > 1)
    count_changeable = sum(df['changeable'])
    print('Частота омонимичных словоформ:\n\t* абсолютная\t%f05\n\t* относительная\t%f05'
          % (count_omonimes/float(count_of_usage), count_omonimes/float(count_changeable)))

    ### Абсолютная и относительная частота словоформ с лексико-морфологической омонимией
    print()
    gr = re.compile(r'=[A-Z]{1,7}[=,]{1}')
    def gr_mystem(x):
        return list(map(lambda x: x.replace('=','').replace(',','').replace(' ',''), gr.findall(x)))

    df['grammar_omo'] = list(map(lambda x: gr_mystem(str(x)), df['spec_y']))
    df['count_grammar'] = list(map(lambda x: len(set(x)),df['grammar_omo']))

    count_of_morphy_lexical_omo = sum(df['count_grammar'] > 2)
    print('Частота словоформ с лексико-морфологической омонимией:\n\t* абсолютная\t%f05\n\t* относительная\t%f05'
          % (count_of_morphy_lexical_omo/float(count_of_usage), count_of_morphy_lexical_omo/float(count_changeable)))

    ###  Максимальное и среднее число омонимов у словоформ текста
    print()
    max_count_of_lexems = max(df['count_of_lexems'])
    mean_count_of_lexems = df[df['count_of_lexems']>1]['count_of_lexems'].mean()
    print('\t* максимальное число омонимов\t%d\n\t* среднее число омонимов\t%f'
          % (max_count_of_lexems,mean_count_of_lexems))

    ### Cловоформы с наибольшим числом омонимов
    print()
    word_form = df[['word_x', 'spec_x']][df['count_of_lexems'] == max_count_of_lexems]
    word_form.columns = ['словоформа','омоним']
    word_form = word_form.reset_index(drop = True)
    print('У этих слов %d омонима:' % max_count_of_lexems)
    pr_head = ['словоформа','омоним']
    pr_tab = list(map(lambda x,y: [x,y], word_form['словоформа'], word_form['омоним']))
    print(tabulate(pr_tab, headers=pr_head))
    print()
    word_form = df[['word_x', 'spec_x']][df['count_of_lexems'] == max_count_of_lexems-1]
    word_form.columns = ['словоформа','омоним']
    word_form = word_form.reset_index(drop = True)
    print('У этих слов %d омонима:' % (max_count_of_lexems-1))
    pr_head = ['словоформа','омоним']
    pr_tab = list(map(lambda x,y: [x,y], word_form['словоформа'], word_form['омоним']))
    print(tabulate(pr_tab, headers=pr_head))
    ### Наиболее частотный омоним
    print()
    omonimes = df[df.count_of_lexems > 1]
    temp = omonimes[['spec_x','word_x','spec_y']]
    temp = pd.DataFrame(temp.groupby(['word_x','spec_x'])['spec_y'].count()).reset_index()
    temp = temp[temp['spec_y']>2].reset_index(drop = True)
    temp = temp.sort_values('spec_y',ascending=False).reset_index(drop = True)
    temp.columns = ['словоформа','вероятные лексемы','количество']
    print('Наиболее частотные омонимы:')
    pr_head = ['словоформа','вероятные лексемы','количество']
    pr_tab = list(map(lambda x,y,z: [x,y,z], temp['словоформа'],temp['вероятные лексемы'],temp['количество']))
    print(tabulate(pr_tab, headers=pr_head))

In [446]:
texts = {'Бродский. Сборник стихов.':'1_brodskiy.txt', #+
         'Бунин. Темные аллеи.':'2_bynin_tyomnye_allei.txt', #+
         'Агния Барто. Стихи.':'3_agniya_barto.txt', #+
         'УК РФ':'4_uk_rf.txt', #+
         'Хэммингуэй. Старик и море.':'5_hemmingyey_starik_i_more.txt' #+
        }

In [447]:
sys.stdout = open('C:\\Users\Ivanitskiy\\Documents\\CL\\report_without_tables.txt', 'w')
for i in texts:
    print(i)
    homework_1(texts.get(i))
    print('*******')